# Coursera Capstone Project-The Battle of Neighborhoods (Week 1)

### The Battle of Neighborhoods (Week 1)

**Gerardo A. Navia - Part 1 : Introduction and Data Sections**

In [158]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Coursera Capstone Week 1 - REPORT CONTENT

1. **Introduction Section** : ⁃ Discussion of the business problem and the interested audience in this project.
2. **Data Section:**
    ⁃ Description of the data that will be used to solve the problem and the sources.

In [2]:
# A description of the problem and a discussion of the background. (15 marks)

## 1. Introduction Section :

### Discussion of the business problem and the audience who would be interested in this project.

#### Description of the Problem and Background

#### Scenario:

I am a Learner data scientist residing in India. I currently live in Kolkata under the state West Bengal with my friend Sumit and we enjoy many ammenities and venues in the area, such as various cousine restaurants, cafes, food shops and entertainment. Sumit has been offered a great opportunity to work in Manhattan, NY. Although, He is a bit stressed about a comfortable place to live in Manhattan. Therefore, I decided to apply the learned skills during the Coursera course to explore ways to make sure my decision is factual and rewarding. Of course, there are alternatives to achieve the answer using available Google and Social media tools, but it rewarding doing it myself with learned tools



#### Business Problem:

The challenge to resolve is being able to find a rental apartment unit in Manhattan NY that offers similar characteristics and benefits to my current situation. Therefore, in order to set a basis for comparison, I want to find a renta unit subject to the following conditions:

1. Apartment with min 2 bedrooms with monthly rent not to exceed US$7000/month
2. Unit located within walking distance (<=1.0 mile, 1.6 km) from a subway metro station in Manhattan
3. Area with ammenities and venues similar to the ones described for current location 

As a reference, I have included a map of venues near current residence.

#### Interested Audience

I believe this is a relevant challenge with valid questions for anyone moving to other large city in US, EU or Asia. The same methodology can be applied in accordance to demands as applicable. This case is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.

In [257]:
# A description of the data and how it will be used to solve the problem. (15 marks)

# 2. Data Section:

## Description of the data and its sources that will be used to solve the problem

### 2.1 Data of Current Situation

I Currently reside in the Park Street area of Kolkata in india. I use Foursquare to identify the venues around the area of residence which are then shown in the India map shown in methodology and execution in section 3.0 . It serves as a reference for comparison with the desired future location in Manhattan NY.

### 2.2 Description of the Data:

The following data is required to answer the issues of the problem:

1. List of Boroughs and neighborhoods of Manhattan with their geodata (latitud and longitud)
2. List of Subway metro stations in Manhattan with their address location
3. List of apartments for rent in Manhattan area with their addresses and price
4. Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc
5. Venues for each Manhattan neighborhood ( than can be clustered)
6. Venues for subway metro stations, as needed

### 2.3 Data Required to resolve the problem

The data will be used as follows:

Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)
Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately
Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.
create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.
The procesing of these DATA will allow to answer the key questions to make a decision:

1. what is the cost of rent (per square ft) around a mile radius from each subway metro station?
2. what is the area of Manhattan with best rental pricing that meets criteria established?
3. What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?
4. What are the venues of the two best places to live? How the prices compare?
5. How venues distribute among Manhattan neighborhoods and around metro stations?
6. Are there tradeoffs between size and price and location?
7. Any other interesting statistical data findings of the real estate and overall data.

# 3.Reference of venues around current residence in kolkata,India for comparison to Manhattan place

In [127]:
!pip install geocoder
import geocoder

In [229]:
address = 'Park Street,India'
#address = 'Mccallum Street, Singapore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of My home are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of My home are 18.5968087, 73.7791717.


In [230]:
neighborhood_latitude=latitude
neighborhood_longitude=longitude

In [231]:
VERSION = '20180605'
CLIENT_ID = 'NXXX11OQJ3F3GZSODERMSQNLIZOJOTATTFXNKFIXCVQUTTRJ'
CLIENT_SECRET = 'QMVFJ2K2OQ5CDUUGB3Q1IPAGWJB20JTFDRYHSA2MV0VDRWDP'
latitude = neighborhood_latitude
longitude = neighborhood_longitude
radius = 900 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NXXX11OQJ3F3GZSODERMSQNLIZOJOTATTFXNKFIXCVQUTTRJ&client_secret=QMVFJ2K2OQ5CDUUGB3Q1IPAGWJB20JTFDRYHSA2MV0VDRWDP&v=20180605&ll=18.5968087,73.7791717&radius=900&limit=100'

In [232]:
results = requests.get(url).json()
#results

In [244]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [256]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues

,name,categories,address,lat,lng
0,Subway,Sandwich Place,NaN,18.602922,73.776887
1,18 Degrees,Lounge,NaN,18.593509,73.785879
2,Hotel Ambience Executive,Hotel,NaN,18.600523,73.775396
3,Spot 18 Mall,Shopping Mall,Pimple Saudagar/Rahatani,18.593470,73.786181
4,KFC Restaurant,American Restaurant,Spot 18 Mall,18.593440,73.786083
5,Fountain inn,Hotel,Kalewadi phata,18.600644,73.780106
6,desi treat,Indian Restaurant,NaN,18.596009,73.780768
7,Medplus,Pharmacy,NaN,18.597140,73.776850
8,Pride Purple Square,Shopping Mall,Kalewadi Phata,18.603311,73.777126
9,Pixel Viilage,Photography Studio,"505 , spot 18 multiplex",18.593479,73.785934


In [258]:

# create map of Singapore place  using latitude and longitude values
map_Kolkata = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_Kolkata)
map_Kolkata